In [15]:
import os, shutil
from keras import layers, models, optimizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [5]:
orginal_dataset_dir = "/home/cj/DL/data/dogs-vs-cats"
orginal_train_dir = os.path.join(orginal_dataset_dir, "train")


os.chmod(orginal_dataset_dir, 0o777)

base_dir = os.path.join(orginal_dataset_dir, 'sub_set')

create_dir_if_not_exist(base_dir)

train_dir = os.path.join(base_dir, "train")

create_dir_if_not_exist(train_dir)

validation_dir = os.path.join(base_dir, "validation")

create_dir_if_not_exist(validation_dir)

test_dir = os.path.join(base_dir, "test")

create_dir_if_not_exist(test_dir)


train_cats_dir = os.path.join(train_dir, "cats")

create_dir_if_not_exist(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, "dogs")

create_dir_if_not_exist(train_dogs_dir)


validation_cats_dir = os.path.join(validation_dir, "cats")

create_dir_if_not_exist(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, "dogs")

create_dir_if_not_exist(validation_dogs_dir)


test_cats_dir = os.path.join(test_dir, "cats")

create_dir_if_not_exist(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, "dogs")

create_dir_if_not_exist(test_dogs_dir)




In [4]:
def create_dir_if_not_exist (url):
    
    if not (os.path.exists(url)):
        os.mkdir(url)

In [41]:


def copy_data_to_correct_dir():
    
    start_index = 0
    
    for i in range(0,3):
        
        
        
        base_url = [train_dogs_dir, train_cats_dir] 
        
        if i == 1:
            base_url = [validation_dogs_dir, validation_cats_dir]
        elif i == 2:
            base_url = [test_dogs_dir, test_cats_dir]
        
        end_index = start_index + (500 if i > 0 else 1000)
        
        for index in range(start_index, end_index):
            
            file_name = f'dog.{index}.jpg'
            src_url = os.path.join(orginal_train_dir, file_name)
            destination_url = os.path.join(base_url[0], file_name)
            
            if not os.path.exists(destination_url):
                shutil.copyfile(src_url, destination_url)
            
            
            file_name = f'cat.{index}.jpg'
            src_url = os.path.join(orginal_train_dir, file_name)
            destination_url = os.path.join(base_url[1], file_name)
        
            if not os.path.exists(destination_url):
                shutil.copyfile(src_url, destination_url)
        
       
       
        start_index = end_index

In [43]:
copy_data_to_correct_dir()

In [46]:
print('total training cat images:', len(os.listdir(train_cats_dir)))

print('total training dog images:', len(os.listdir(train_dogs_dir)))

print('total validation cat images:', len(os.listdir(validation_cats_dir)))

print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

print('total test cat images:', len(os.listdir(test_cats_dir)))

print('total test dog images:', len(os.listdir(test_dogs_dir)))

total training cat images: 1000
total training dog images: 1000
total validation cat images: 500
total validation dog images: 500
total test cat images: 500
total test dog images: 500


In [49]:
model1 = models.Sequential();
model1.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)))
model1.add(layers.MaxPooling2D((2,2)))
model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2,2)))
model1.add(layers.Conv2D(128, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2,2)))
model1.add(layers.Conv2D(128, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2,2)))
model1.add(layers.Flatten())
model1.add(layers.Dense(512, activation='relu'))
model1.add(layers.Dense(1, activation='sigmoid'))

In [50]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 17, 17, 128)     

In [52]:
model1.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=["accuracy"])

/home/linuxbrew/.linuxbrew/opt/python@3.10/lib/python3.10/site-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [55]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [56]:
history = model1.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=50
)

/tmp/ipykernel_113722/1820336663.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model1.fit_generator(


Epoch 1/30


2023-09-30 14:31:33.394209: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-09-30 14:31:36.806677: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 56074240 exceeds 10% of free system memory.
2023-09-30 14:31:37.856823: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 56074240 exceeds 10% of free system memory.


  1/100 [..............................] - ETA: 7:36 - loss: 0.6908 - accuracy: 0.6000

2023-09-30 14:31:38.105948: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 56074240 exceeds 10% of free system memory.
2023-09-30 14:31:39.033978: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 56074240 exceeds 10% of free system memory.


  2/100 [..............................] - ETA: 1:56 - loss: 0.8055 - accuracy: 0.5250

2023-09-30 14:31:39.298354: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 56074240 exceeds 10% of free system memory.


100/100 [==============================] - ETA: 0s - loss: 0.6940 - accuracy: 0.5270

2023-09-30 14:33:15.445432: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


100/100 [==============================] - 114s 1s/step - loss: 0.6940 - accuracy: 0.5270 - val_loss: 0.6893 - val_accuracy: 0.5220
Epoch 2/30
100/100 [==============================] - 108s 1s/step - loss: 0.6603 - accuracy: 0.6165 - val_loss: 0.6486 - val_accuracy: 0.6100
Epoch 3/30
100/100 [==============================] - 110s 1s/step - loss: 0.6262 - accuracy: 0.6415 - val_loss: 0.6152 - val_accuracy: 0.6560
Epoch 4/30
100/100 [==============================] - 108s 1s/step - loss: 0.5847 - accuracy: 0.6855 - val_loss: 0.6049 - val_accuracy: 0.6640
Epoch 5/30
100/100 [==============================] - 107s 1s/step - loss: 0.5410 - accuracy: 0.7235 - val_loss: 0.6064 - val_accuracy: 0.6640
Epoch 6/30
100/100 [==============================] - 108s 1s/step - loss: 0.5185 - accuracy: 0.7310 - val_loss: 0.5594 - val_accuracy: 0.7110
Epoch 7/30
100/100 [==============================] - 108s 1s/step - loss: 0.4816 - accuracy: 0.7725 - val_loss: 0.5646 - val_accuracy: 0.6960
Epoch 8/30

<h1>DATA AUGMENTATION</h1>

In [11]:
aug_train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

aug_test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
aug_train_generator = aug_train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

aug_validation_generator = aug_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [7]:
model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)))
model2.add(layers.MaxPooling2D((2,2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2,2)))
model2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2,2)))
model2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2,2)))
model2.add(layers.Flatten())
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(512, activation='relu'))
model2.add(layers.Dense(1, activation='sigmoid'))

2023-09-30 20:47:45.397665: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [9]:
model2.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=["acc"]
)

In [14]:
history2 = model2.fit(
    aug_train_generator,
    epochs=30,
    validation_data=aug_validation_generator
)

Epoch 1/30


2023-09-30 21:19:45.568210: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


63/63 [==============================] - ETA: 0s - loss: 0.6780 - acc: 0.5755

2023-09-30 21:21:20.864175: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


63/63 [==============================] - 107s 2s/step - loss: 0.6780 - acc: 0.5755 - val_loss: 0.6854 - val_acc: 0.5250
Epoch 2/30
63/63 [==============================] - 105s 2s/step - loss: 0.6707 - acc: 0.5875 - val_loss: 0.6817 - val_acc: 0.5470
Epoch 3/30
63/63 [==============================] - 105s 2s/step - loss: 0.6658 - acc: 0.5955 - val_loss: 0.7457 - val_acc: 0.5070
Epoch 4/30
63/63 [==============================] - 106s 2s/step - loss: 0.6580 - acc: 0.6110 - val_loss: 0.6457 - val_acc: 0.6010
Epoch 5/30
63/63 [==============================] - 106s 2s/step - loss: 0.6495 - acc: 0.6105 - val_loss: 0.6183 - val_acc: 0.6640
Epoch 6/30
63/63 [==============================] - 108s 2s/step - loss: 0.6429 - acc: 0.6210 - val_loss: 0.5994 - val_acc: 0.6730
Epoch 7/30
63/63 [==============================] - 105s 2s/step - loss: 0.6238 - acc: 0.6535 - val_loss: 0.6594 - val_acc: 0.5930
Epoch 8/30
63/63 [==============================] - 106s 2s/step - loss: 0.6102 - acc: 0.6525 

<h1>USING PRETRAINED MODEL (FEATURE EXTRACTION)</h1>

In [17]:
pretrain_data_gen = ImageDataGenerator(rescale=1./255)
batch_size = 20



In [19]:
from keras.applications import VGG16

conv_base = VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)

58889256/58889256 [==============================] - 31s 1us/step


In [20]:
def extract_features(directory, sample_count):
    
    features = np.zeros((sample_count, 4, 4, 512))
    labels = np.zeros((sample_count))
    
    generator = pretrain_data_gen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary'
    )
    
    i = 0
    
    for batch_inputs, batch_labels in generator:
        
        batch_features = conv_base.predict(batch_inputs)
        features[i*batch_size: (i+1)*batch_size] = batch_features
        labels[i*batch_size: (i+1)*batch_size] = batch_labels
        i+=1
        
        if i * batch_size >= sample_count:
            break
    return features, labels

In [21]:
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
1/1 [==============================] - 3s 3s/step
Found 1000 images belonging to 2 classes.
1/1 [==============================] - 3s 3s/step
Found 1000 images belonging to 2 classes.
1/1 [==============================] - 3s 3s/step


1/1 [==============================] - 3s 3s/step


In [22]:
model3 = models.Sequential()
model3.add(layers.Flatten(input_shape=(4,4,512)))
model3.add(layers.Dense(256, activation="relu"))
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(1, activation="sigmoid"))

In [23]:
model3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 256)               2097408   
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2,097,665
Trainable params: 2,097,665
Non-trainable params: 0
_________________________________________________________________


In [31]:
model3.compile(
    optimizer=optimizers.RMSprop(learning_rate=2e-5),
    loss='binary_crossentropy',
    metrics=['acc']
)

In [28]:
train_features.shape

(2000, 4, 4, 512)

In [32]:
history3 = model3.fit(
    train_features,
    train_labels,
    epochs=30,
    batch_size=20,
    validation_data=(validation_features, validation_labels)
)

Epoch 1/30
100/100 [==============================] - 6s 43ms/step - loss: 0.6133 - acc: 0.6540 - val_loss: 0.4545 - val_acc: 0.8190
Epoch 2/30
100/100 [==============================] - 4s 40ms/step - loss: 0.4305 - acc: 0.8025 - val_loss: 0.3800 - val_acc: 0.8430
Epoch 3/30
100/100 [==============================] - 4s 39ms/step - loss: 0.3687 - acc: 0.8440 - val_loss: 0.3485 - val_acc: 0.8480
Epoch 4/30
100/100 [==============================] - 4s 40ms/step - loss: 0.3209 - acc: 0.8680 - val_loss: 0.3108 - val_acc: 0.8800
Epoch 5/30
100/100 [==============================] - 4s 40ms/step - loss: 0.2951 - acc: 0.8725 - val_loss: 0.2995 - val_acc: 0.8850
Epoch 6/30
100/100 [==============================] - 4s 40ms/step - loss: 0.2678 - acc: 0.9010 - val_loss: 0.2890 - val_acc: 0.8840
Epoch 7/30
100/100 [==============================] - 4s 39ms/step - loss: 0.2434 - acc: 0.9065 - val_loss: 0.2725 - val_acc: 0.8910
Epoch 8/30
100/100 [==============================] - 4s 40ms/step - 